# Simple network topology

The network topology consists of one AP called ap1 and two stations, sta1 and sta2. Their IP and MAC addresses can be found below.

![title](imgs/simple.png)

**Note**: the code you will find below can be simplified through the following command: `sudo mn --wifi --mac --arp --ap bmv2 --client-isolation --json-file=default`

In [1]:
#!/usr/bin/python

import os
import sys

from mininet.log import setLogLevel, info
from mn_wifi.cli import CLI
from mn_wifi.net import Mininet_wifi
from mn_wifi.bmv2 import P4AP


def topology():
    'Create a network.'
    net = Mininet_wifi()

    info('*** Adding stations\n')
    sta1 = net.addStation('sta1', ip='10.0.0.1', mac="00:00:00:00:00:01")
    sta2 = net.addStation('sta2', ip='10.0.0.2', mac="00:00:00:00:00:02")

    path = os.path.dirname(os.getcwd())
    json_file = path + '/p4-scenarios/simple.json'
    
    info('*** Adding P4APs\n')
    ap1 = net.addAccessPoint('ap1', cls=P4AP, mac="00:00:00:00:00:03",
                             client_isolation=True, netcfg=True, 
                             thriftport=50001, json=json_file)

    net.configureWifiNodes()

    info('*** Creating links\n')
    net.addLink(sta1, ap1)
    net.addLink(sta2, ap1)
    
    info('*** Starting network\n')
    net.start()
    net.staticArp()

    info('*** Running CLI\n')
    CLI(net)

    info('*** Stopping network\n')
    net.stop()


if __name__ == '__main__':
    setLogLevel('info')
    topology()

*** Adding stations
*** Adding P4APs
*** Creating links
*** Starting network
*** Starting controller(s)

*** Starting L2 nodes
ap1 

..⚡️ simple_switch_grpc @ 13634 thrift @ 50001



*** Running CLI
*** Starting CLI:


mininet-wifi>  ap1 simple_switch_CLI --thrift-port 50001 <<< "show_tables"	


Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: MyIngress.ipv4_lpm             [implementation=None, mk=ipv4.dstAddr(exact, 32)]
RuntimeCmd: 


...

Obtaining JSON from switch...
Done
Control utility for runtime P4 table manipulation
RuntimeCmd: Adding entry to exact match table MyIngress.ipv4_lpm
match key:           EXACT-0a:00:00:02
action:              ipv4_forward
runtime data:        00:00:00:00:00:02	00:01
Entry has been added with handle 1
RuntimeCmd: 


mininet-wifi> sta1 ping -c1 sta2


PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.
64 bytes from 10.0.0.2: icmp_seq=1 ttl=63 time=0.641 ms

--- 10.0.0.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.641/0.641/0.641/0.000 ms


mininet-wifi> exit


*** Stopping network
*** Stopping 0 controllers

*** Stopping 2 links
..
*** Stopping switches/access points
ap1 .
*** Stopping nodes
sta1 sta2 

*** Removing WiFi module and Configurations
*** Killing mac80211_hwsim

*** Done


**Great work!**